# City and Country
The purpose of this notebook is to parse through location text to retireve city, and country data, which will later be used for analysis

In [0]:
%sql
-- Create silver-level locations dimension table
CREATE OR REPLACE TABLE silver.dim_locations AS
SELECT
    DENSE_RANK() OVER (ORDER BY location) AS location_key,
    location
FROM (
    SELECT DISTINCT location
    FROM silver.jobs_table
    WHERE location IS NOT NULL
)
;


In [0]:
from pyspark.sql import SparkSession
import pandas as pd
from geotext import GeoText
import pycountry

spark = SparkSession.builder.getOrCreate()

# Load silver.dim_locations
silver_df = spark.table("silver.dim_locations").toPandas()

# Drop existing gold.dim_locations if it already exists, then create a new one
spark.sql("DROP TABLE IF EXISTS gold.dim_locations")
gold_df.write.saveAsTable("gold.dim_locations")

# Helper Functo to extract city and country from the location string
def extract_city_country(location):
    places = GeoText(location)
    city = places.cities[0] if places.cities else None
    country = places.countries[0] if places.countries else None
    if country:
        try:
            country = pycountry.countries.lookup(country).name
        except LookupError:
            pass
    return pd.Series([city, country])

silver_df[['city', 'country']] = silver_df['location'].apply(extract_city_country)

# Fill in blanks
silver_df['city'] = silver_df['city'].fillna('Unknown')
silver_df['country'] = silver_df['country'].fillna('Unknown')

# Convert back to Spark 
gold_df = spark.createDataFrame(silver_df)

gold_df.write.mode("overwrite").option("mergeSchema", "true").saveAsTable("gold.dim_locations")
